# Распознавание объектов на фотографиях



# Цель
Распознавание объектов на фотографиях (Object Recognition in Photographs)
CIFAR-10 (классификация небольших изображений по десяти классам: самолет,
автомобиль, птица, кошка, олень, собака, лягушка, лошадь, корабль и грузовик).

# Задачи

* Ознакомиться со сверточными нейронными сетями
* Изучить построение модели в Keras в функциональном виде
* Изучить работу слоя разреживания (Dropout)

# Выполнение работы


### Подключение модулей
Набор данных CIFRA-10 уже входит в состав Keras в форме набора из четырех массивов Numpy.

In [1]:
import sys
import tensorflow as tf
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD


Напишем функцию для загрузки тренеровочных и тестовых данных

In [2]:
def load_dataset():
    # Загружаем данные
    (trainX, trainY), (testX, testY) = cifar10.load_data()
   
    # кодируем метки
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    
    return trainX, trainY, testX, testY

Создадим функцию которая преобразует наши данные в необходимы маштаб

In [3]:
def prep_pixels(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    return train_norm, test_norm

Создадим модель обучения с Dropout

In [4]:
def define_model_with_Drop():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Создадим модель обучения без Dropout

In [5]:
def define_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
   
    return model

Создадим функию отрисовки

In [6]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    pyplot.show()
    
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    pyplot.show()

Функция запуска нашей нейроной сети

In [7]:
def run_test_harness(d):
    trainX, trainY, testX, testY = load_dataset()
    trainX, testX = prep_pixels(trainX, testX)
    
    if d == 0:
        model = define_model()
    else:
        model = define_model_with_Drop()
    
    #Обцчение сети
    history = model.fit(trainX, trainY, epochs=50, batch_size=64, validation_data=(testX, testY), verbose=1)
    
    #Оценка модели
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f' % (acc * 100.0))
   
    #Отрисовка
    summarize_diagnostics(history)

Запустим модель с Dropout

In [ ]:
run_test_harness(1)

Epoch 1/50


Запустим модель без Dropout

In [ ]:
run_test_harness(0)